In [117]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

from ipywidgets import fixed
import ipywidgets as widgets

from cannybase import get_edge_data

In [150]:
def plot_contour(data, contour, plotter=None, spacing=None):
    if plotter is None:
        plotter = pv.Plotter()
    else:
        plotter.clear()
    mesh = pv.wrap(data)
    if spacing:
        mesh.spacing = spacing
    contour = mesh.contour([contour])
    light = pv.Light(position=(10, 10, 10), focal_point=(0, 0, 0), color='white', intensity=1.0)
    light2 = pv.Light(position=(-10, -10, -10), focal_point=(0, 0, 0), color='white', intensity=1.0)
    light3= pv.Light(position=(10, 0, 0), focal_point=(0, 0, 0), color='white', intensity=1.0)
    light4= pv.Light(position=(0, 0, 10), focal_point=(0, 0, 0), color='white', intensity=1.0)
    plotter.add_mesh(mesh.outline())
    plotter.add_light(light)
    plotter.add_light(light2)
    plotter.add_light(light3)
    plotter.add_light(light4)
    plotter.add_mesh(contour, show_scalar_bar=False)
    return plotter

In [119]:
def interact_contour_edge_data(data, thresholds, min_axes, gaussian, contour, plotter, spacing=None):
    print(f'\rUpdating with {min_axes=}, {thresholds=}, {gaussian=}, {contour=}...', end='')
    edge_data = get_edge_data(data, thresholds[0], thresholds[1], min_axes, gaussian)
    plot_contour(edge_data, contour, plotter, spacing)
    print(f'\rUpdated with {min_axes=}, {thresholds=}, {gaussian=}, {contour=}. Number of edge points: {edge_data.size - np.isnan(edge_data).sum()}.', end='')

In [120]:
def make_widgets(con_min = 0, con_max = 255):
    threshold_widget = widgets.IntRangeSlider(
        min=0,
        max=255,
        value=(255,255),
        description='Canny Thresholds'
    )
    min_axes_widget = widgets.IntSlider(
        min=0,
        max=3,
        description='Min Axes'
    )
    gaussian_widget = widgets.FloatSlider(
        min=0,
        max=7,
        value=0,
        description='Gaussian SD'
    )
    contour_widget = widgets.FloatSlider(
        min=con_min,
        max=con_max,
        value=50,
        description='Contour Value'
    )
    return threshold_widget, min_axes_widget, gaussian_widget, contour_widget

## `HeadMRVolume.raw`

In [121]:
headmr_data = np.fromfile('./local-data/HeadMRVolume.raw', dtype='uint8')
headmr_data = headmr_data.reshape(42, 62, 48)

In [ ]:
threshold_widget, min_axes_widget, gaussian_widget, contour_widget = make_widgets()
pl = plot_contour(headmr_data, 50)
widgets.interact(
    interact_contour_edge_data, 
    data=fixed(headmr_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    contour=contour_widget,
    plotter=fixed(pl), 
    spacing=fixed(None)
)
pl.show()

interactive(children=(IntRangeSlider(value=(255, 255), description='Canny Thresholds', max=255), IntSlider(val…

Widget(value='<iframe src="http://localhost:61952/index.html?ui=P_0x1de333d12b0_76&reconnect=auto" class="pyvi…

# `T2.raw`

In [123]:
t2_data = np.fromfile('./local-data/T2.raw', dtype='float32')
t2_data = t2_data.reshape(320, 320, 256)

In [146]:
threshold_widget, min_axes_widget, gaussian_widget, contour_widget = make_widgets(np.min(t2_data), np.max(t2_data))
pl = plot_contour(t2_data, 50)
widgets.interact(
    interact_contour_edge_data, 
    data=fixed(t2_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    contour=contour_widget,
    plotter=fixed(pl), 
    spacing=fixed(None)
)
pl.show()

interactive(children=(IntRangeSlider(value=(255, 255), description='Canny Thresholds', max=255), IntSlider(val…

Widget(value='<iframe src="http://localhost:61952/index.html?ui=P_0x1ddc2d624e0_77&reconnect=auto" class="pyvi…

# `ctscan_ez.vtk`

In [105]:
ctscan = pv.read('./local-data/ctscan_ez.vtk')
ctscan_data = ctscan.point_data['image_data'].reshape(ctscan.dimensions[2], ctscan.dimensions[1], ctscan.dimensions[0])
ctscan_data = np.ndarray.copy(np.swapaxes(ctscan_data, 0, 2))

In [151]:
threshold_widget, min_axes_widget, gaussian_widget, contour_widget = make_widgets(np.min(ctscan_data), np.max(ctscan_data))
pl = plot_contour(ctscan_data, spacing=ctscan.spacing, contour=50)
widgets.interact(
    interact_contour_edge_data, 
    data=fixed(ctscan_data), 
    thresholds=threshold_widget, 
    min_axes=min_axes_widget, 
    gaussian=gaussian_widget,
    contour=contour_widget,
    plotter=fixed(pl), 
    spacing=fixed(ctscan.spacing)
)
pl.show()

interactive(children=(IntRangeSlider(value=(255, 255), description='Canny Thresholds', max=255), IntSlider(val…

Widget(value='<iframe src="http://localhost:61952/index.html?ui=P_0x1dd15897bf0_80&reconnect=auto" class="pyvi…